# 3. Train-Predict RF
Result:
- Kaggle score:

Reference:
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
- http://scikit-learn.org/stable/modules/model_evaluation.html#model-evaluation

## Run name

In [1]:
import time
import os
import pandas as pd
import gc

project_name = 'Google_LandMark_Rec'
step_name = 'RF'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: Google_LandMark_Rec_RF_20180321_141332


## 项目文件夹

In [2]:
cwd = os.getcwd()
data1_folder = '/data1/kaggle/landmark-recognition-challenge/'
input_folder = os.path.join(data1_folder, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t' + input_folder)
print('output_folder: \t' + output_folder)
print('model_folder: \t' + model_folder)
print('feature_folder: \t' + feature_folder)
print('post_pca_feature_folder: \t' + post_pca_feature_folder)
print('log_folder: \t' + log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

input_folder: 	/data1/kaggle/landmark-recognition-challenge/input
output_folder: 	/data2/kaggle/landmark-recognition-challenge/output
model_folder: 	/data2/kaggle/landmark-recognition-challenge/model
feature_folder: 	/data2/kaggle/landmark-recognition-challenge/feature
post_pca_feature_folder: 	/data2/kaggle/landmark-recognition-challenge/post_pca_feature
log_folder: 	/data2/kaggle/landmark-recognition-challenge/log


In [3]:
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## 加载feature

In [4]:
%%time
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2018)

x_data = []
y_data = {}
x_test = []


image_size = 150
time_str = '20180311-151108'
cwd = os.getcwd()
# feature_cgg16 = os.path.join(cwd, 'feature', 'feature_VGG16_{}.h5'.format(20180219))
# feature_cgg19 = os.path.join(cwd, 'feature', 'feature_VGG19_{}.h5'.format(20180219))
# feature_resnet50 = os.path.join(cwd, 'feature', 'feature_ResNet50_{}.h5'.format(20180220))
# feature_xception = os.path.join(cwd, 'feature', 'feature_Xception_{}.h5'.format(20180221))
feature_inceptionV3 = os.path.join(cwd, 'post_pca_feature', 'post_pca_feature_InceptionV3_%s_%s.h5' % (image_size, time_str))
# feature_inceptionResNetV2 = os.path.join(cwd, 'feature', 'feature_InceptionResNetV2_%s.h5' % time_str)
# for filename in [feature_cgg16, feature_cgg19, feature_resnet50, feature_xception, feature_inception, feature_inceptionResNetV2]:
for filename in [feature_inceptionV3]:
    with h5py.File(filename, 'r') as h:
        x_data.append(np.array(h['train']))
        y_data = np.array(h['train_labels'])
#         x_val.append(np.array(h['val']))
#         y_val = np.array(h['val_labels'])
        x_test.append(np.array(h['test']))

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


CPU times: user 1.94 s, sys: 4.24 s, total: 6.17 s
Wall time: 13.7 s


In [5]:
print(x_data[0].shape)
print(len(y_data))
print(x_test[0].shape)

(1219426, 32)
1219426
(115942, 32)


In [6]:
%%time
x_data = np.concatenate(x_data, axis=-1)
x_test = np.concatenate(x_test, axis=-1)
print(x_data.shape)
print(x_test.shape)

(1219426, 32)
(115942, 32)
CPU times: user 92 ms, sys: 140 ms, total: 232 ms
Wall time: 231 ms


In [7]:
# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.0025, random_state=5)

# print(x_train.shape)
# print(x_val.shape)
# print(y_train.shape)
# print(y_val.shape)

## Train KNN with GridSearchCV

In [8]:
set_y_data = list(set(y_data))
print(len(set_y_data))

14951


In [9]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.01, random_state=2017)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_test.shape)


(1207231, 32)
(12195, 32)
(1207231,)
(12195,)
(115942, 32)


In [10]:
set_y_train = list(set(y_train))
print(len(set_y_train))

14950


In [11]:
%%time
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=50,
    min_samples_split=5,
    min_samples_leaf=10,
    max_depth=20, 
    random_state=0, 
    n_jobs=50,
    verbose=1
)
clf.fit(x_data[:50000], y_data[:50000])

[Parallel(n_jobs=50)]: Done   2 out of  50 | elapsed:   32.7s remaining: 13.1min


CPU times: user 28min 18s, sys: 18.5 s, total: 28min 37s
Wall time: 37.1 s


[Parallel(n_jobs=50)]: Done  50 out of  50 | elapsed:   37.0s finished


In [ ]:
%%time
from sklearn.metrics import accuracy_score
y_data_pred = clf.predict(x_data[:50000])
print(y_data_pred.shape)

In [ ]:
print(clf.feature_importances_)

print(accuracy_score(y_data[:50000], y_data_pred))

In [ ]:
print(clf.n_classes_)

In [ ]:
print(clf.classes_)

In [ ]:
%%time
import time
import os

selected_clf_name = 'RandomForestClassifier'
clf = clfs[selected_clf_name]
run_name_acc = run_name + '_' + str(int(preds[selected_clf_name]*10000)).zfill(4)
print(run_name_acc)

pred_file = os.path.join(output_folder, run_name_acc + '.csv')
print(pred_file)

## Predict

In [ ]:
%%time
y_test = clf.predict(x_test)
print(y_test.shape)

In [ ]:
print(clf.best_estimator_)

In [ ]:
# a = list(set(y_test))
print(len(a))
print(a[:10])

In [ ]:
%%time
y_test = clf.predict_proba(x_test)
print(y_test.shape)

In [ ]:
print(y_test[0])

In [ ]:
sample_submission_csv = pd.read_csv(sample_submission_folder)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

In [ ]:
# 这里证明os.listdir()得到的图片名称list不正确
files = os.listdir(os.path.join(input_folder, 'data_test', 'test'))
print(files[:10])

In [ ]:
# 这里证明ImageDataGenerator()得到的图片名称list才是正确
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator()
image_size = (299, 299)
batch_size = 128
test_generator  = gen.flow_from_directory(test_folder, image_size, shuffle=False, batch_size=batch_size)
print('test_generator')
print(len(test_generator.filenames))
print(test_generator.filenames[:10])

In [ ]:
%%time
max_indexes = np.argmax(y_test, -1)
print(max_indexes.shape)

test_dict = {}
for i, paire in enumerate(zip(test_generator.filenames, max_indexes)):
    image_name, label = paire[0], paire[1]
    image_id = image_name[5:-4]
    test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])
    test_dict[image_id] = '%d %.4f' % (label, 1)

#确认图片的id是否能与ImageDataGenerator()对应上
for key in list(test_dict.keys())[:10]:
    print('%s  %s' % (key, test_dict[key]))

In [ ]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = sample_submission_csv.iloc[i, 0]
#     landmarks = sample_submission_csv.iloc[i, 1]
    if image_id in test_dict:
        pred_landmarks = test_dict[image_id]
#         print('%s  %s' % (image_id, pred_landmarks))
        sample_submission_csv.iloc[i, 1] = pred_landmarks
    else:
#         print(image_id)
#         sample_submission_csv.iloc[i, 1] = '9633 1.0' # 属于9633的类最多，所以全都设置成这个类，可能会比设置成空得到的结果好
        sample_submission_csv.iloc[i, 1] = '' # 设置成空
    count += 1
    if count % 10000 == 0:
        print(int(count/10000), end=' ')
display(sample_submission_csv.head(2))

In [ ]:
pred_file = os.path.join(output_folder, 'pred_' + run_name_acc + '.csv')
sample_submission_csv.to_csv(pred_file, index=None)

In [ ]:
print(run_name_acc)
print('Done !')